In [1]:
import numpy as np
from matplotlib import pyplot as plt
import timeit

%matplotlib inline
%load_ext autoreload
%autoreload 2

In [2]:
import h5py
from pathlib import Path
from FKMC.jobmanager import *

In [3]:
config = dict(
              mcmc_routine='quantum_cython_mcmc',
              N_steps = 5,
              bins = 1,
              N_burn_in = 5,
              N_system = np.array([4,8,16]),
              mu = 0,
              beta = np.array([1,2,3]),
              U = np.array([1,2,3]),
              V = -0.1*np.array([1,2,3]),
              alpha= 1.5,
              outer_loop = np.array(['N_system','V'], dtype=h5py.special_dtype(vlen=str)),
              inner_loop = np.array(['U','beta'], dtype=h5py.special_dtype(vlen=str)),
              quantum = True,
              output_history = True,
              output_correlator = False,
              output_state = False,
            )

working_dir = Path('../data/test')
setup_mcmc(config, working_dir, overwrite = True)

config = read_config_file(working_dir)
config

INFO:FKMC.jobmanager:Working in: ../data/test


KeyError: 'output_corrrelator'

In [ ]:
%%time
from FKMC.jobmanager import outer_loop_shape

working_dir = Path('../data/test')
config = read_config_file(working_dir)
outer_job_length = outer_loop_shape(config).prod()

for job_id in range(outer_job_length):
    run_mcmc(job_id, working_dir = working_dir, overwrite = True)

In [ ]:
#the problem is that the correlator is a different shape for each value of N_system,

import logging
logging.getLogger().setLevel(logging.DEBUG)

gather_mcmc(working_dir, do_all = False)